In [1]:
import pandas as pd
import numpy as np

In [2]:
#Creating Content Based Movie Recommendation System Using NLP TFI-DF on Overview of Movie
#Reading the files from the google Drive
credits = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Content Based/tmdb_5000_credits.csv', sep = ",", engine='python')
movies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Content Based/tmdb_5000_movies.csv', sep=",", engine='python')


In [3]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [5]:
#Rename and Merging credits and movies on id
credits_column_renamed = credits.rename(columns={"movie_id" : "id"})
movies_credits_merge = movies.merge(credits_column_renamed,on="id")
movies_credits_merge.head(2)
#movies_credits_merge["overview"].isna().sum()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [6]:
#Dropping unneccesary columns
movies_cleaned_df = movies_credits_merge.drop(columns=["homepage","title_y","status","production_countries"])
movies_cleaned_df = movies_cleaned_df.rename(columns={"title_x":"title"})
movies_cleaned_df["overview"] = movies_cleaned_df["overview"].fillna("")
movies_cleaned_df.head(2)

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [7]:
movies_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   id                    4803 non-null   int64  
 3   keywords              4803 non-null   object 
 4   original_language     4803 non-null   object 
 5   original_title        4803 non-null   object 
 6   overview              4803 non-null   object 
 7   popularity            4803 non-null   float64
 8   production_companies  4803 non-null   object 
 9   release_date          4802 non-null   object 
 10  revenue               4803 non-null   int64  
 11  runtime               4801 non-null   float64
 12  spoken_languages      4803 non-null   object 
 13  tagline               3959 non-null   object 
 14  title                 4803 non-null   object 
 15  vote_average         

In [8]:
movies_cleaned_df["overview"][2]

'A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.'

In [9]:
#Cleaning the Data
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords                    #is,am,a,are,the....
#from nltk.stem import PorterStemmer                 #jumping->jump,battled->battle....
from nltk.stem import WordNetLemmatizer              #rocks->rock,better->Good....

sw = set(stopwords.words("english"))
#ps = PorterStemmer()
lm = WordNetLemmatizer() 

def clean_overview(sentence):
  sentence = sentence.lower()   
  sentence = re.sub("[^a-zA-Z]+"," ",sentence)          #Removing which are not characters like numbers,',' etc.

  sentence = sentence.split()
  sentence = [lm.lemmatize(s) for s in sentence if s not in sw]       #Removing Stopwords and apply WordNetLemmatizer by iterating on a sentence
  sentence = " ".join(sentence)
  
  return sentence

clean_overview(movies_cleaned_df["overview"][2])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'cryptic message bond past sends trail uncover sinister organization battle political force keep secret service alive bond peel back layer deceit reveal terrible truth behind spectre'

In [10]:
movies_cleaned_df["overview"] = movies_cleaned_df["overview"].apply(clean_overview)

In [11]:
movies_cleaned_df.head(5)

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,nd century paraplegic marine dispatched moon p...,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,captain barbossa long believed dead come back ...,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,cryptic message bond past sends trail uncover ...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,following death district attorney harvey dent ...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,john carter war weary former military captain ...,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [12]:
#Create Vector of matrix for Each Movie because of pair wise similarity
#Now using TF-IDF which helps to create document matrix from sentences
#TF-IDF(Term Frequency- Inverse Document Frequency) explaination
#TF*IDF = (No. of rep. of words in sentence/No. of words in sentence) * log(No. of Sentences/No. of sentences containing words)

from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3, max_features=None,
                      ngram_range = (1,3) )                        #take combination of 1-3 different words

In [13]:
#Fitting the TF.IDF on the 'overview'
tfv_marix = tfv.fit_transform(movies_cleaned_df["overview"])        #convert into sparx matrix which have lots of 0 values

In [14]:
tfv_marix

<4803x10271 sparse matrix of type '<class 'numpy.float64'>'
	with 136587 stored elements in Compressed Sparse Row format>

In [15]:
tfv_marix.shape                                   #rows and features(Columns)

(4803, 10271)

In [33]:
from sklearn.metrics.pairwise import sigmoid_kernel

#Compute Sigmoid Kernel                           
sig = sigmoid_kernel(tfv_marix,tfv_marix)                    #Perform Sigmoid with Every row("overview") or combination of matrix's
with open('sigdata.txt','w+') as f:  
  sig.tofile(f)
  for line in f.readlines():
    print(line)
print('done')

done


In [17]:
sig[5]

array([0.76159519, 0.76159416, 0.76159416, ..., 0.76159509, 0.76159436,
       0.76159437])

In [18]:
#Mapping of indices and movie titles
indices = pd.Series(movies_cleaned_df.index,index=movies_cleaned_df['original_title']).drop_duplicates()

In [19]:
indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [20]:
indices['Spectre']

2

In [21]:
sig[2]

array([0.76159416, 0.76159494, 0.76163504, ..., 0.76159476, 0.76159416,
       0.76159416])

In [22]:
list(enumerate(sig[indices['Spectre']]))        #list of range Values

[(0, 0.7615941559557649),
 (1, 0.7615949360198039),
 (2, 0.7616350422571493),
 (3, 0.7615941559557649),
 (4, 0.7615941559557649),
 (5, 0.7615941559557649),
 (6, 0.7615941559557649),
 (7, 0.7615954649394991),
 (8, 0.7615950516163807),
 (9, 0.7615941559557649),
 (10, 0.7615941559557649),
 (11, 0.761601384922797),
 (12, 0.7615941559557649),
 (13, 0.76159460238889),
 (14, 0.7615941559557649),
 (15, 0.7615941559557649),
 (16, 0.7615947607843407),
 (17, 0.7615960919482768),
 (18, 0.761597064093079),
 (19, 0.7615941559557649),
 (20, 0.7615941559557649),
 (21, 0.7615941559557649),
 (22, 0.761594749690804),
 (23, 0.7615949775694483),
 (24, 0.7615941559557649),
 (25, 0.7615941559557649),
 (26, 0.7615949613869737),
 (27, 0.7615946243261349),
 (28, 0.7615941559557649),
 (29, 0.7615997079019453),
 (30, 0.7615941559557649),
 (31, 0.7615941559557649),
 (32, 0.7615941559557649),
 (33, 0.7615941559557649),
 (34, 0.7615941559557649),
 (35, 0.7615987995071747),
 (36, 0.7615947438757356),
 (37, 0.76159415

In [23]:
sorted(list(enumerate(sig[indices['Spectre']])),key=lambda x:x[1],reverse=True)

[(2, 0.7616350422571493),
 (1343, 0.76160495348053),
 (4071, 0.7616033461687902),
 (3162, 0.7616031258406848),
 (11, 0.761601384922797),
 (3351, 0.7616007003665644),
 (1808, 0.7616006487703091),
 (1200, 0.761600270511196),
 (1281, 0.7616000654962034),
 (2008, 0.7616000408184724),
 (1717, 0.7616000061169125),
 (218, 0.761599985553369),
 (1599, 0.7615997908124055),
 (29, 0.7615997079019453),
 (4339, 0.7615990382955555),
 (4328, 0.761598987814532),
 (3336, 0.7615988869343884),
 (3285, 0.7615988134495701),
 (4402, 0.761598800389366),
 (35, 0.7615987995071747),
 (4684, 0.7615987540134985),
 (1837, 0.761598625373958),
 (2080, 0.7615985979692068),
 (1743, 0.7615985704985321),
 (367, 0.7615985103347455),
 (1131, 0.7615984710258932),
 (277, 0.7615984386454282),
 (188, 0.761598397228596),
 (3969, 0.7615980793592845),
 (147, 0.7615980585005697),
 (3610, 0.7615980162206446),
 (1374, 0.7615979361332574),
 (170, 0.7615979019315298),
 (3310, 0.7615978870438531),
 (1642, 0.7615978185082525),
 (2326, 0

In [24]:
def recommendation(title,sig=sig):

  #Get the index corresponding to original_title
  idx = indices[title]
  
  #Get the Paiswise similarity Scores
  sig_scores = list(enumerate(sig[idx]))  

  #Sort the MOvies in Descending Order
  sig_scores = sorted(sig_scores,key=lambda x:x[1],reverse=True)

  #Score of the 10 most similar Movies
  sig_scores = sig_scores[1:11]

  #Movie indices
  movies_indices = [i[0] for i in sig_scores]

  #Top 10 most similar movies
  return movies_cleaned_df['original_title'].iloc[movies_indices]

In [25]:
recommendation("Avatar")

1341                Obitaemyy Ostrov
3604                       Apollo 18
634                       The Matrix
775                        Supernova
2130                    The American
311     The Adventures of Pluto Nash
529                 Tears of the Sun
847                         Semi-Pro
1449                       The Order
1345                  Dolphin Tale 2
Name: original_title, dtype: object